# 3 非对称量化
原始集合空间的定义域被映射到量化空间中是非对称的。

<img src="./img/3_1_asym.png" alt="非对称量化" width="600" />

相较于对称量化，非对称量化的映射范围只与最大值和最小值有关，与绝对值无关，因此非对称量化能够最大程度的利用目标集合定义域的全部动态范围，并且非对称量化的区间分布能够与原始集合保持一致（因此非对称量化常常用于激活值量化，特别是当激活函数是ReLu的情况，对称量化常用于权重量化）



In [2]:
import torch

In [4]:
# 以对称量化中的数据为例,定义一个tensor
x = torch.tensor([5.47,3.08,-7.59,0.00,-1.95,-4.57,10.80])
print(f"{x},{x.dtype}")

tensor([ 5.4700,  3.0800, -7.5900,  0.0000, -1.9500, -4.5700, 10.8000]),torch.float32


非对称量化的计算公式

<img src="./img/3_2_asym.png" alt="非对称量化" width="600" />

In [8]:
# 1. 计算缩放因子，同样考虑INT8作为目标集合
x_max = x.max()
x_min = x.min()
q_max = 127
q_min = -128
s = (x_max - x_min) / (q_max - q_min)
print(f"x_max = {x_max} \n x_min = {x_min} \n q_max = {q_max} \n q_min = {q_min} \n s = {s}")

x_max = 10.800000190734863 
 x_min = -7.590000152587891 
 q_max = 127 
 q_min = -128 
 s = 0.0721176415681839


In [11]:
# 2. 计算零点，零点其是本质上是代表目标集合定义域相对于原始集合定义域的偏移
z_0 = q_min - x_min/s   # 四舍五入前
z = torch.round(z_0)    # 四舍五入后
print(f"z_0 = {z_0} \n z = {z}") 

z_0 = -22.755294799804688 
 z = -23.0


In [ ]:
# 3. 对原始集合进行非对称量化
q = torch.round(x / s + z).to(torch.int8)  # 四舍五入，并转成整型
q

tensor([  53,   20, -128,  -23,  -50,  -86,  127], dtype=torch.int8)

In [17]:
# 4. 反量化
x_hat = s * (q - z)
print(f"{x_hat},{x_hat.dtype}")

tensor([ 5.4809,  3.1011, -7.5724,  0.0000, -1.9472, -4.5434, 10.8176]),torch.float32


In [ ]:
# 查看量化误差
err = x - x_hat
print(f"x={x}")
print(f"x_hat={x_hat}")
print(f"err={err}")

x=tensor([ 5.4700,  3.0800, -7.5900,  0.0000, -1.9500, -4.5700, 10.8000])
x_hat=tensor([ 5.4809,  3.1011, -7.5724,  0.0000, -1.9472, -4.5434, 10.8176])
err=tensor([-0.0109, -0.0211, -0.0176,  0.0000, -0.0028, -0.0266, -0.0176])


对称量化和非对称量化映射前后的区别:

<img src="./img/3_3_asym.png" alt="" width="600" />

## 关于异常值

此处我们讨论异常值对量化影响。

In [29]:
# 异常值是指原始数据中出现的离群值，通常会导致缩放因子和零点的计算异常
# 定义一个包含异常值的tensor
x = torch.tensor([-0.59,-0.21,-0.07,-0.13,0.28,0.57,256])
print(f"{x},{x.dtype}")

tensor([-5.9000e-01, -2.1000e-01, -7.0000e-02, -1.3000e-01,  2.8000e-01,
         5.7000e-01,  2.5600e+02]),torch.float32


In [35]:
# 量化目标为INT8，对称量化之后的情况
a = x.abs().max()
q_max_val = 127
s_sym = a / q_max_val
q_sym = x / s_sym
q_sym = torch.round(q_sym.to(torch.int8))
print(f"对原始数据进行对称量化：\n 缩放因子s = {s_sym} \n 对称量化后输出：\n {q_sym}")

对原始数据进行对称量化：
 缩放因子s = 2.0157480239868164 
 对称量化后输出：
 tensor([  0,   0,   0,   0,   0,   0, 127], dtype=torch.int8)


进行对称量化的前后情况如图所示。受异常值影响，原本在0点附近的值被全部压缩到0点。

<img src="./img/3_4_asym.png" alt="" width="600" />

异常值通常会导致推理过程中的激活值异常，为了避免这种情况的发生，可以通过裁剪动态范围，使其异常值映射到一个固定的范围中。


In [38]:
# 手动设置动态范围，将超出范围的值裁剪到范围内
# 量化目标为INT8，对称量化之后的情况
x = torch.clip(x,-5,5)
a = x.abs().max()
q_max_val = 127
s_sym = a / q_max_val
q_sym = x / s_sym
q_sym = torch.round(q_sym.to(torch.int8))
print(f"对原始数据进行对称量化：\n 缩放因子s = {s_sym} \n 对称量化后输出：\n {q_sym}")


对原始数据进行对称量化：
 缩放因子s = 0.03937007859349251 
 对称量化后输出：
 tensor([-14,  -5,  -1,  -3,   7,  14, 127], dtype=torch.int8)


进行裁剪之后输出情况如图：

<img src="./img/3_5_asym.png" alt="" width="600" />


通过裁剪，非异常值的量化误差会显著减少，但是异常值的量化误差会增加。